In [3]:
import json
import numpy as np

In [4]:
def MVP(self, u, d):
    """
    Solves dx = f(x, u, d)

    Parameters
    ----------
    u : int or float 
        Insulin injection rate.
    d : int or float 
        Meal ingestion rate.
    
    Returns
    -------
    dx : numpy array
        Solution to system of differential equations. 
    """
    dD1 = d - self.D1/self.taum
    dD2 = (self.D1 - self.D2)/self.taum
    dIsc = u/(self.tau1 * self.C1) - self.Isc/self.tau1
    dIp = (self.Isc - self.Ip)/self.tau2
    dIeff = -self.p2 * self.Ieff + self.p2 * self.S1 * self.Ip
    dG = - (self.gezi + self.Ieff) * self.G + self.egp0 + 1000 * self.D2 / (self.Vg * self.taum)
    dGsc = (self.G - self.Gsc) / self.timestep

    dx = np.array([dD1, dD2, dIsc, dIp, dIeff, dG, dGsc])
    return dx


def EHM(self, uI, d, uG = 0):
    """
    Solves dx = f(x, u, d)

    Parameters
    ----------
    u : int or float 
        Insulin injection rate.
    d : int or float 
        Meal ingestion rate.
    
    Returns
    -------
    dx : numpy array
        Solution to system of differential equations. 
    """
    G = self.Q1/self.VG
    RA = self.f*self.D2/self.TauD
    D = 1000 * d/self.MwG
    HRmax = 220 - self.age
    HR = self.HRR * (HRmax - self.HR0) + self.HR0
    fE1x = self.E1/(self.a * self.HR0)**self.n
    fE1 = fE1x/(1+fE1x)
    F01c = min(self.F01, self.F01 * G / 4.5)
    FR = max(0.003 * (G - 9) * self.VG, 0) 

    dx = []
    dx.append((G - self.G)/self.TauIG)
    dx.append(-F01c - FR - self.x1 * self.Q1 + self.k12 * self.Q2 + RA + self.EGP0 * (1 - self.x3) \
        + self.Kglu * self.VG * self.Z2 - self.alpha * self.E2**2 * self.x1 * self.Q1)
    dx.append(self.x1 * self.Q1 - (self.k12 + self.x2) * self.Q2 + self.alpha * self.E2**2 * self.x1 * self.Q1 \
        - self.alpha * self.E2**2 * self.x2 * self.Q2 - self.beta * self.E1 / self.HR0)

    dx.append(uI - self.S1 / self.TauS)
    dx.append((self.S1 - self.S2)/self.TauS)
    dx.append(self.S2 / (self.VI * self.TauS) - self.ke * self.I)
    dx.append(self.kb1 * self.I - self.ka1 * self.x1)
    dx.append(self.kb2 * self.I - self.ka2 * self.x2)
    dx.append(self.kb3 * self.I - self.ka3 * self.x3)
    dx.append(self.AG * D - self.D1 / self.TauD)
    dx.append((self.D1 - self.D2)/self.TauD)
    dx.append(uG - self.Z1 / self.Tauglu)
    dx.append((self.Z1 - self.Z2)/self.Tauglu)
    dx.append((HR - self.HR0 - self.E1)/self.TauHR)
    dx.append(-(fE1/self.Tauin - 1/self.TE) * self.E2 + fE1 * self.TE /(self.c1 + self.c2))
    dx.append((self.c1 * fE1 + self.c2 - self.TE)/self.Tauex)
    return np.array(dx)

In [2]:
class Healthy:
    def __init__(self, model = "EHM", **kwargs):
        self.model = model.upper()
        
        defaults = {} #tomt dictionary 
        with open('config.json', 'r') as f:
            data = json.load(f) #læs json-fil
        defaults.updata(data["general"]) #tilføj "general" til dictionary(defaults)
        defaults.update(data[self.model]) #tilføj modelegenskaberne til dictionary(defaults)
        defaults.update(kwargs) #tilføj keywordarguments til dictionary(defaults)

        if self.model == "MVP":
            self.f_func = lambda *args: MVP(self,*args) #caller MVP-modellen
        else:
            self.f_func = lambda *args: EHM(self,*args) #caller EHM-modellen
        self.pancreas = PKPM(timestep=self.timestep)
        
    def PKPM(self, G): #Pancrease-model/ Bugespytkirtlen (giver insulin)
        if G <= self.Gu:
            alpha1 = self.alpha1 #Transcription rate constant
            alpha2 = 0 #Response of glucose stimulus
            delta1 = self.delta1 #mRNA degradation rate constant
            f1 = self.fb 
            v = self.v #Biosynthesis rate of proinsulin aggregates
        else:
            alpha1 = 116.98 #Transcription rate constant
            delta1 = 1.5e-4 #mRNA degradation rate constant
            v = 7.89e-5 #Biosynthesis rate of proinsulin aggregates
            f1 = self.fb + (1 - self.fb) *  (G - self.Gu) / (self.Kf +  G - self.Gu) 
            if G <= self.Gl:
                alpha2 = self.hhat * (G - self.Gu)/(G - self.Gu) #Response of glucose stimulus
            else:
                alpha2 = self.hhat #Response of glucose stimulus

        #Implementering af Differentialligninger
        dM = alpha1 - delta1 * self.M #Insulin mRNA concentration
        dP = v * self.M - self.delta2 * self.P - self.k * self.P * self.rho * self.DIR #Proinsulin
        dR =  self.k * self.P * self.rho * self.DIR - self.gamma * self.R #Reserve granule
        dgamma = self.eta * (-self.gamma + self.gammab + alpha2) #ATP/ADP
        dD = self.gamma * self.R - self.k1p * (self.CT - self.DIR) * self.D - self.k1m * self.DIR - self.rho * self.DIR #Docked granule
        dDIR = self.k1p * (self.CT - self.DIR) * self.D - self.k1m * self.DIR - self.rho * self.DIR #Immediately releasable granule
        drho = self.zeta * (self.rho + self.rhob + self.krho * (self.gamma - self.gammab)) #Ca^(2+)
        return np.array([dM, dP, dR, dgamma, dD, dDIR, drho])

In [ ]:
class TD1:
    def __init__(self, model ="EHM", **kwargs):
        self.model = model.upper()
        
        defaults = {} #tomt dictionary 
        with open('config.json', 'r') as f:
            data = json.load(f) #læs json-fil
        defaults.updata(data["general"]) #tilføj "general" til dictionary(defaults)
        defaults.update(data[self.model]) #tilføj modelegenskaberne til dictionary(defaults)
        defaults.update(kwargs) #tilføj keywordarguments til dictionary(defaults)

        if self.model == "MVP":
            self.f_func = lambda *args: MVP(self,*args) #caller MVP-modellen
        else:
            self.f_func = lambda *args: EHM(self,*args) #caller EHM-modellen

    def PID_controller(self, I, y, y_prev):
        """
        :input I: Integral term
        :input Gbar: Glucose concentration target
        :input y: Current blood glucose
        :input y_prev: Previous blood glucose
        :input us: Insulin steady state
        Tuning parameters
        :input Kp: Range 0-0.5
        :input Ti: 100-1000 minutes
        :input Td : 0-30 min
        """
        ek = y - self.Gbar
        Pk = self.Kp * ek
        Ki = self.Kp * self.timestep / self.Ti
        Ikp1 = I + Ki * ek
        Kd = self.Kp * self.Td / self.timestep
        Dk = Kd * (y - y_prev)
        uk = self.us + Pk + I + Dk
        uk = max(uk, 0)
        
        return uk, Ikp1


In [ ]:
class TD2:
    def __init__(self, model ="EHM", **kwargs):
        self.model = model.upper()
        
        defaults = {} #tomt dictionary 
        with open('config.json', 'r') as f:
            data = json.load(f) #læs json-fil
        defaults.updata(data["general"]) #tilføj "general" til dictionary(defaults)
        defaults.update(data[self.model]) #tilføj modelegenskaberne til dictionary(defaults)
        defaults.update(kwargs) #tilføj keywordarguments til dictionary(defaults)

        if self.model == "MVP":
            self.f_func = lambda *args: MVP(self,*args) #caller MVP-modellen
        else:
            self.f_func = lambda *args: EHM(self,*args) #caller EHM-modellen

    def PID_controller(self, I, y, y_prev):
        """
        :input I: Integral term
        :input Gbar: Glucose concentration target
        :input y: Current blood glucose
        :input y_prev: Previous blood glucose
        :input us: Insulin steady state
        Tuning parameters
        :input Kp: Range 0-0.5
        :input Ti: 100-1000 minutes
        :input Td : 0-30 min
        """
        ek = y - self.Gbar
        Pk = self.Kp * ek
        Ki = self.Kp * self.timestep / self.Ti
        Ikp1 = I + Ki * ek
        Kd = self.Kp * self.Td / self.timestep
        Dk = Kd * (y - y_prev)
        uk = self.us + Pk + I + Dk
        uk = max(uk, 0)
        
        return uk, Ikp1
    
    def PKPM(self, G): #Pancrease-model/ Bugespytkirtlen (giver insulin)
        if G <= self.Gu:
            alpha1 = self.alpha1 #Transcription rate constant
            alpha2 = 0 #Response of glucose stimulus
            delta1 = self.delta1 #mRNA degradation rate constant
            f1 = self.fb 
            v = self.v #Biosynthesis rate of proinsulin aggregates
        else:
            alpha1 = 116.98 #Transcription rate constant
            delta1 = 1.5e-4 #mRNA degradation rate constant
            v = 7.89e-5 #Biosynthesis rate of proinsulin aggregates
            f1 = self.fb + (1 - self.fb) *  (G - self.Gu) / (self.Kf +  G - self.Gu) 
            if G <= self.Gl:
                alpha2 = self.hhat * (G - self.Gu)/(G - self.Gu) #Response of glucose stimulus
            else:
                alpha2 = self.hhat #Response of glucose stimulus

        #Implementering af Differentialligninger
        dM = alpha1 - delta1 * self.M #Insulin mRNA concentration
        dP = v * self.M - self.delta2 * self.P - self.k * self.P * self.rho * self.DIR #Proinsulin
        dR =  self.k * self.P * self.rho * self.DIR - self.gamma * self.R #Reserve granule
        dgamma = self.eta * (-self.gamma + self.gammab + alpha2) #ATP/ADP
        dD = self.gamma * self.R - self.k1p * (self.CT - self.DIR) * self.D - self.k1m * self.DIR - self.rho * self.DIR #Docked granule
        dDIR = self.k1p * (self.CT - self.DIR) * self.D - self.k1m * self.DIR - self.rho * self.DIR #Immediately releasable granule
        drho = self.zeta * (self.rho + self.rhob + self.krho * (self.gamma - self.gammab)) #Ca^(2+)
        return np.array([dM, dP, dR, dgamma, dD, dDIR, drho])
